In [ ]:
import pandas as pd

# Cargar los datos
df = pd.read_csv('/content/flights_final.csv')

df.head(2)


,Source Airport Code,Source Airport Name,Source Airport City,Source Airport Country,Source Airport Latitude,Source Airport Longitude,Destination Airport Code,Destination Airport Name,Destination Airport City,Destination Airport Country,Destination Airport Latitude,Destination Airport Longitude
0,COK,Cochin International Airport,Kochi,India,10.152000,76.401901,SHJ,Sharjah International Airport,Sharjah,United Arab Emirates,25.328600,55.517200
1,GIG,Rio Galeão – Tom Jobim International Airport,Rio De Janeiro,Brazil,-22.809999,-43.250557,BSB,Presidente Juscelino Kubistschek International...,Brasilia,Brazil,-15.869167,-47.920834


Funcion

In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict
import heapq

class Graph:
    def __init__(self):
        self.parent = {}
        self.rank = {}

    def make_set(self, vertice):
        self.parent[vertice] = vertice
        self.rank[vertice] = 0

    def find_set(self, vertice):
        if self.parent[vertice] != vertice:
            self.parent[vertice] = self.find_set(self.parent[vertice])  # Path compression
        return self.parent[vertice]

    def union(self, u, v):
        root_u = self.find_set(u)
        root_v = self.find_set(v)

        if root_u != root_v:
            if self.rank[root_u] > self.rank[root_v]:
                self.parent[root_v] = root_u
            else:
                self.parent[root_u] = root_v
                if self.rank[root_u] == self.rank[root_v]:
                    self.rank[root_v] += 1

    # Algoritmo de Kruskal
    def kruskal(self, componente, aristas_componente):
        resultante = []  # Aristas del MST
        cont = 0  # Contador de pasos

        # Inicializar cada vértice en su propio conjunto
        for vertice in componente:
            self.make_set(vertice)

        # Ordenar las aristas por peso
        ordenada = list(aristas_componente)
        ordenada.sort()  # Ordenar por el primer elemento (peso)

        # Recorrer las aristas en orden
        for dato in ordenada:
            peso, u, v = dato
            if self.find_set(u) != self.find_set(v):
                resultante.append(dato)
                cont += 1
                self.union(u, v)

        return resultante

    # Función para hacer DFS en el grafo
    def dfs(self, v, visited, grafo):
        visited.add(v)
        for u in grafo[v]:
            if u not in visited:
                self.dfs(u, visited, grafo)

    # Función para determinar si el grafo es conexo y encontrar componentes
    def es_conexo(self, grafo):
        # Crear un conjunto para marcar los vértices visitados
        visited = set()
        componentes = []

        # Recorrer todos los vértices del grafo
        for vertice in grafo:
            if vertice not in visited:
                # Realizar DFS desde un vértice no visitado
                componente_actual = set()
                self.dfs(vertice, componente_actual, grafo)
                componentes.append(componente_actual)
                visited.update(componente_actual)

        # Si solo hay una componente, el grafo es conexo
        es_conexo = len(componentes) == 1

        return es_conexo, componentes

def dijkstra(graph, start):
    # Inicializamos las distancias para todos los nodos
    distances = {vertex: float('inf') for vertex in graph}
    distances[start] = 0 # Distancia 0 para el nodo inicial
    prio_queue = [(0, start)]  # Cola de prioridad (distancia,nodo)




    while prio_queue:
        # Obtener el nodo que tenga la distancia mas corta
        current_distance, current_vertex = heapq.heappop(prio_queue)

        if current_distance > distances[current_vertex]:
            continue


        # Obtener los nodos adyacentes y los pesos en las aristas entre el nodo actual y el adyacente
        for adj, weight in graph[current_vertex]:
            #Calculamos la distancia que vamos a comparar
            new_distance = current_distance + weight

            # Revisamos si esta nueva distancia es mejor que la que tiene el nodo adyacente actual
            if new_distance < distances[adj]:
                #Actualizamos
                distances[adj] = new_distance

                heapq.heappush(prio_queue, (new_distance, adj))

    return distances

def create_graph(edge_list):
    graph = {}
    for weight, start, end in edge_list:
        if start not in graph:
            graph[start] = []
        if end not in graph:
            graph[end] = []
        graph[start].append((end, weight))

        graph[end].append((start, weight))
    return graph

def get_path(previous_nodes, start, end):
    path = []

    while end is not None:

        path.append(end)
        end = previous_nodes[end]
    path.reverse()
    return path


GRAFO COMPLETO Y GEOLOCALIZACION


In [ ]:
import pandas as pd
import folium
from math import radians, sin, cos, sqrt, atan2
import networkx as nx

# Función para calcular la distancia Haversine entre dos puntos geográficos
def calcular_distancia_haversine(lat1, lon1, lat2, lon2):
    radio_tierra = 6371  # Radio de la Tierra en kilómetros
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    diferencia_lat = lat2 - lat1
    diferencia_lon = lon2 - lon1
    a = sin(diferencia_lat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(diferencia_lon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return radio_tierra * c

# Clase para representar el grafo de aeropuertos
class GrafoAeropuertos:
    def __init__(self):
        self.grafo = nx.Graph()  # Grafo no dirigido y ponderado
        self.aeropuertos = {}

    def agregar_aeropuerto(self, codigo, nombre, ciudad, pais, lat, lon):
        if codigo not in self.aeropuertos:
            self.aeropuertos[codigo] = {
                'nombre': nombre,
                'ciudad': ciudad,
                'pais': pais,
                'latitud': lat,
                'longitud': lon
            }
            self.grafo.add_node(codigo)

    def agregar_ruta(self, codigo_origen, codigo_destino):
        if codigo_origen in self.aeropuertos and codigo_destino in self.aeropuertos:
            aeropuerto_origen = self.aeropuertos[codigo_origen]
            aeropuerto_destino = self.aeropuertos[codigo_destino]
            lat_origen, lon_origen = aeropuerto_origen['latitud'], aeropuerto_origen['longitud']
            lat_destino, lon_destino = aeropuerto_destino['latitud'], aeropuerto_destino['longitud']

            # Calcular la distancia entre los dos aeropuertos
            distancia = calcular_distancia_haversine(lat_origen, lon_origen, lat_destino, lon_destino)

            # Agregar una arista con peso al grafo
            self.grafo.add_edge(codigo_origen, codigo_destino, weight=distancia)

    def mostrar_grafo_en_mapa(self):
        # Crear un mapa centrado en una ubicación genérica
        mapa = folium.Map(location=[0, 0], zoom_start=2)

        # Agregar los aeropuertos y las rutas en el mapa
        for aeropuerto_codigo in self.grafo.nodes:
            aeropuerto_info = self.aeropuertos[aeropuerto_codigo]
            folium.Marker(
                location=[aeropuerto_info['latitud'], aeropuerto_info['longitud']],
                popup=f"{aeropuerto_info['nombre']} ({aeropuerto_codigo})",
                icon=folium.Icon(color="blue")
            ).add_to(mapa)

        for origen, destino, data in self.grafo.edges(data=True):
            lat_origen = self.aeropuertos[origen]['latitud']
            lon_origen = self.aeropuertos[origen]['longitud']
            lat_destino = self.aeropuertos[destino]['latitud']
            lon_destino = self.aeropuertos[destino]['longitud']
            distancia = data['weight']

            folium.PolyLine(
                locations=[(lat_origen, lon_origen), (lat_destino, lon_destino)],
                color='blue',
                weight=2.5,
                opacity=0.8,
                tooltip=f"Distancia: {distancia:.2f} km"
            ).add_to(mapa)

        # Guardar el mapa en un archivo HTML
        mapa.save("mapa_rutas_aereas.html")
        print("Mapa del grafo guardado como 'mapa_rutas_aereas.html'.")

# Leer los datos de los aeropuertos desde un archivo CSV
def construir_grafo_aeropuertos(archivo_csv):
    df = pd.read_csv(archivo_csv)
    grafo_aeropuertos = GrafoAeropuertos()

    for index, row in df.iterrows():
        # Agregar aeropuertos como nodos
        grafo_aeropuertos.agregar_aeropuerto(
            codigo=row['Source Airport Code'],
            nombre=row['Source Airport Name'],
            ciudad=row['Source Airport City'],
            pais=row['Source Airport Country'],
            lat=row['Source Airport Latitude'],
            lon=row['Source Airport Longitude']
        )

        grafo_aeropuertos.agregar_aeropuerto(
            codigo=row['Destination Airport Code'],
            nombre=row['Destination Airport Name'],
            ciudad=row['Destination Airport City'],
            pais=row['Destination Airport Country'],
            lat=row['Destination Airport Latitude'],
            lon=row['Destination Airport Longitude']
        )

        # Agregar rutas como aristas
        grafo_aeropuertos.agregar_ruta(row['Source Airport Code'], row['Destination Airport Code'])

    return grafo_aeropuertos

# MAIN
if __name__ == "__main__":
    archivo_csv = 'flights_final.csv'
    grafo_aeropuertos = construir_grafo_aeropuertos(archivo_csv)

    # Mostrar el grafo en un mapa
    grafo_aeropuertos.mostrar_grafo_en_mapa()


Mapa del grafo guardado como 'mapa_rutas_aereas.html'.


In [ ]:
import pandas as pd
import folium

# Función para cargar los datos de los aeropuertos
def cargar_datos_aeropuertos(csv_filename):
    df = pd.read_csv(csv_filename)
    info_aeropuertos = {}

    # Recorrer las filas del archivo CSV y almacenar la información de cada aeropuerto
    for index, row in df.iterrows():
        codigo = row['Source Airport Code']
        info_aeropuertos[codigo] = {
            'name': row['Source Airport Name'],
            'city': row['Source Airport City'],
            'country': row['Source Airport Country'],
            'lat': float(row['Source Airport Latitude']),
            'lon': float(row['Source Airport Longitude'])
        }

    return info_aeropuertos

# Función para mostrar los aeropuertos en el mapa
def mostrar_geolocalizacion_aeropuertos(info_aeropuertos):
    # Crear un mapa centrado en una ubicación genérica (centrado en la Tierra)
    mapa = folium.Map(location=[0, 0], zoom_start=2)

    # Recorrer los aeropuertos y agregar marcadores con la información
    for codigo, info in info_aeropuertos.items():
        nombre = info['name']
        ciudad = info['city']
        pais = info['country']
        lat = info['lat']
        lon = info['lon']

        # Añadir marcador al mapa
        folium.Marker(
            location=[lat, lon],
            popup=f"{nombre} ({codigo})<br>Ciudad: {ciudad}<br>País: {pais}",
            icon=folium.Icon(color='blue')
        ).add_to(mapa)

    # Guardar el mapa en un archivo HTML
    mapa.save("geolocalizacion_aeropuertos.html")
    print("Mapa con la geolocalización de los aeropuertos guardado como 'geolocalizacion_aeropuertos.html'.")

# MAIN
if __name__ == "__main__":
    # Cargar los datos de los aeropuertos desde el archivo CSV
    archivo_csv = 'flights_final.csv'
    info_aeropuertos = cargar_datos_aeropuertos(archivo_csv)

    # Mostrar la geolocalización de los aeropuertos en el mapa
    mostrar_geolocalizacion_aeropuertos(info_aeropuertos)


Mapa con la geolocalización de los aeropuertos guardado como 'geolocalizacion_aeropuertos.html'.


PUNTO 1 Y 2



In [ ]:
import pandas as pd
import numpy as np
import folium
from collections import defaultdict
from math import radians, sin, cos, sqrt, atan2

# Función para calcular la distancia Haversine entre dos puntos geográficos
def calcular_distancia_haversine(lat1, lon1, lat2, lon2):
    radio_tierra = 6371  # Radio de la Tierra en kilómetros
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return radio_tierra * c

# Clase Grafo para manejar el MST
class Graph:
    def __init__(self):
        self.parent = {}
        self.rank = {}

    def make_set(self, vertice):
        self.parent[vertice] = vertice
        self.rank[vertice] = 0

    def find(self, vertice):
        if self.parent[vertice] != vertice:
            self.parent[vertice] = self.find(self.parent[vertice])
        return self.parent[vertice]

    def union(self, vertice1, vertice2):
        root1 = self.find(vertice1)
        root2 = self.find(vertice2)

        if root1 != root2:
            if self.rank[root1] > self.rank[root2]:
                self.parent[root2] = root1
            else:
                self.parent[root1] = root2
                if self.rank[root1] == self.rank[root2]:
                    self.rank[root2] += 1

    # Algoritmo de Kruskal para obtener el MST
    def kruskal(self, vertices, aristas):
        mst = []

        # Crear un conjunto para cada vértice
        for vertice in vertices:
            self.make_set(vertice)

        # Ordenar las aristas por peso
        aristas_ordenadas = sorted(aristas, key=lambda x: x[0])

        for arista in aristas_ordenadas:
            peso, vertice1, vertice2 = arista
            if self.find(vertice1) != self.find(vertice2):
                self.union(vertice1, vertice2)
                mst.append(arista)

        return mst

# Función para graficar el MST en un mapa usando Folium
def mostrar_mst_en_mapa(mst, aeropuertos):
    # Crear un mapa centrado en una ubicación genérica
    mapa = folium.Map(location=[0, 0], zoom_start=2)

    # Agregar las rutas del MST (solo aristas, sin marcadores)
    for (peso, u, v) in mst:
        lat_u = aeropuertos[u]['lat']
        lon_u = aeropuertos[u]['lon']
        lat_v = aeropuertos[v]['lat']
        lon_v = aeropuertos[v]['lon']

        # Dibujar la arista del MST
        folium.PolyLine(
            locations=[(lat_u, lon_u), (lat_v, lon_v)],
            color='green',
            weight=2.5,
            opacity=0.8,
            tooltip=f"Distancia: {peso:.2f} km"
        ).add_to(mapa)

    # Guardar el mapa en un archivo HTML
    mapa.save("mst_map.html")
    print("Mapa del MST guardado como 'mst_map.html'.")

# Leer los datos del CSV
df = pd.read_csv('flights_final.csv')

# Crear un mapeo de códigos de aeropuerto a enteros (si los códigos no son numéricos)
airport_codes = pd.concat([df['Source Airport Code'], df['Destination Airport Code']]).unique()
code_to_int = {code: idx for idx, code in enumerate(airport_codes)}
int_to_code = {idx: code for code, idx in code_to_int.items()}

# Crear el grafo usando las rutas de vuelos (grafo no dirigido)
grafo = defaultdict(list)
aristas = []  # Lista para almacenar todas las aristas con sus pesos
aeropuertos = {}  # Información de los aeropuertos

# Agregar las aristas (aeropuertos con la distancia entre ellos)
for index, row in df.iterrows():
    source_code = code_to_int[row['Source Airport Code']]
    dest_code = code_to_int[row['Destination Airport Code']]
    source_lat = row['Source Airport Latitude']
    source_lon = row['Source Airport Longitude']
    dest_lat = row['Destination Airport Latitude']
    dest_lon = row['Destination Airport Longitude']

    # Calcular la distancia entre los aeropuertos usando Haversine
    distancia = calcular_distancia_haversine(source_lat, source_lon, dest_lat, dest_lon)

    # Agregar la arista al grafo
    grafo[source_code].append(dest_code)
    grafo[dest_code].append(source_code)

    # Agregar la arista con peso a la lista de aristas
    aristas.append((distancia, source_code, dest_code))

    # Guardar la información de los aeropuertos
    aeropuertos[source_code] = {
        'name': row['Source Airport Name'],
        'lat': source_lat,
        'lon': source_lon
    }
    aeropuertos[dest_code] = {
        'name': row['Destination Airport Name'],
        'lat': dest_lat,
        'lon': dest_lon
    }

# Inicializar la clase Graph
graph_obj = Graph()

# Función para encontrar componentes conexas
def encontrar_componentes_conexas(grafo):
    visitado = set()
    componentes = []

    def dfs(v, componente_actual):
        visitado.add(v)
        componente_actual.add(v)
        for u in grafo[v]:
            if u not in visitado:
                dfs(u, componente_actual)

    for vertice in grafo:
        if vertice not in visitado:
            componente_actual = set()
            dfs(vertice, componente_actual)
            componentes.append(componente_actual)

    return componentes

# Encontrar las componentes conexas
componentes = encontrar_componentes_conexas(grafo)

# Mostrar el resultado de la conectividad
print("El grafo es NO conexo.")
print(f"El grafo tiene un total de {len(componentes)} componentes conexas.")
for i, componente in enumerate(componentes):
    print(f"Componente {i+1} tiene {len(componente)} vértices.")

# Inicializar el MST total para todas las componentes
mst_total = []

# Ahora calculamos el MST para cada componente
for i, componente in enumerate(componentes):
    # Filtrar las aristas que pertenecen a esta componente
    aristas_componente = [(peso, u, v) for (peso, u, v) in aristas if u in componente and v in componente]

    # Aplicar Kruskal para esta componente
    mst = graph_obj.kruskal(componente, aristas_componente)
    mst_total.extend(mst)

    # Calcular el peso total del MST de esta componente
    peso_total = sum(peso for peso, _, _ in mst)

    # Mostrar resultados
    print(f"Peso total del Árbol de Expansión Mínima (MST) para la Componente {i+1}: {peso_total:.2f} km")

# Graficar el MST total (de todas las componentes) en un solo mapa
mostrar_mst_en_mapa(mst_total, aeropuertos)



El grafo es NO conexo.
El grafo tiene un total de 7 componentes conexas.
Componente 1 tiene 3230 vértices.
Componente 2 tiene 4 vértices.
Componente 3 tiene 4 vértices.
Componente 4 tiene 10 vértices.
Componente 5 tiene 4 vértices.
Componente 6 tiene 2 vértices.
Componente 7 tiene 2 vértices.
Peso total del Árbol de Expansión Mínima (MST) para la Componente 1: 1197584.25 km
Peso total del Árbol de Expansión Mínima (MST) para la Componente 2: 1598.90 km
Peso total del Árbol de Expansión Mínima (MST) para la Componente 3: 787.25 km
Peso total del Árbol de Expansión Mínima (MST) para la Componente 4: 1441.06 km
Peso total del Árbol de Expansión Mínima (MST) para la Componente 5: 255.94 km
Peso total del Árbol de Expansión Mínima (MST) para la Componente 6: 98.79 km
Peso total del Árbol de Expansión Mínima (MST) para la Componente 7: 37.83 km
Mapa del MST guardado como 'mst_map.html'.


PUNTO 3

In [ ]:
import heapq

# Crear un diccionario con la información de los aeropuertos
airport_info = {}
for index, row in df.iterrows():
    code = str(row['Source Airport Code'])
    airport_info[code] = {
        'name': str(row['Source Airport Name']),
        'city': str(row['Source Airport City']),
        'country': str(row['Source Airport Country']),
        'lat': str(row['Source Airport Latitude']),
        'lon': str(row['Source Airport Longitude']),
    }

# Solicitar al usuario el código de un aeropuerto
codigo = input("Escriba el código de algún aeropuerto: ").strip()

# Crear el grafo con las aristas
graph2 = create_graph(aristas)

# Buscar el aeropuerto en el DataFrame y calcular las distancias desde este aeropuerto
for index, row in df.iterrows():
    if codigo == str(row['Source Airport Code']):
        # Extraer la información del aeropuerto inicial
        name = str(row['Source Airport Name'])
        city = str(row['Source Airport City'])
        country = str(row['Source Airport Country'])
        lat = str(row['Source Airport Latitude'])
        lon = str(row['Source Airport Longitude'])

        # Convertir el código del aeropuerto a su valor entero (para trabajar con el grafo)
        c = code_to_int[codigo]

        # Ejecutar Dijkstra para encontrar los caminos mínimos desde este aeropuerto
        dist = dijkstra(graph2, c)
        break

# Mostrar la información del aeropuerto inicial
print("\nInformación del punto inicial:")
print(f"Código: {codigo}")
print(f"Nombre: {name}")
print(f"Ciudad: {city}")
print(f"País: {country}")
print(f"Latitud: {lat}")
print(f"Longitud: {lon}")
print("-" * 50)

# Filtrar los aeropuertos alcanzables y ordenar por los más lejanos (caminos mínimos más largos)
filtered_data = {k: v for k, v in dist.items() if v != float('inf')}
longest_paths = heapq.nlargest(10, filtered_data.items(), key=lambda item: item[1])

# Convertir los índices enteros de vuelta a los códigos de aeropuerto
int_to_code = {idx: code for code, idx in code_to_int.items()}

# Mostrar la información de los 10 aeropuertos más lejanos
print("\nInformación de destinos más lejanos de los caminos mínimos:")
for cde, distancia in longest_paths:
    converted = int_to_code.get(cde, "Code not found")

    if converted in airport_info:
        info = airport_info[converted]
        name = info['name']
        city = info['city']
        country = info['country']
        lat = info['lat']
        lon = info['lon']

        # Mostrar la información del aeropuerto
        print(f"Código: {converted}")
        print(f"Nombre: {name}")
        print(f"Ciudad: {city}")
        print(f"País: {country}")
        print(f"Latitud: {lat}")
        print(f"Longitud: {lon}")
        print(f"Distancia desde {codigo}: {distancia:.2f} km")
        print("-" * 50)


Escriba el código de algún aeropuerto: SDQ

Información del punto inicial:
Código: SDQ
Nombre: Las Américas International Airport
Ciudad: Santo Domingo
País: Dominican Republic
Latitud: 18.42970085
Longitud: -69.66889954
--------------------------------------------------

Información de destinos más lejanos de los caminos mínimos:
Código: CCK
Nombre: Cocos (Keeling) Islands Airport
Ciudad: Cocos Keeling Island
País: Cocos (Keeling) Islands
Latitud: -12.18830013
Longitud: 96.83390045
Distancia desde SDQ: 23737.32 km
--------------------------------------------------
Código: XCH
Nombre: Christmas Island Airport
Ciudad: Christmas Island
País: Christmas Island
Latitud: -10.45059967
Longitud: 105.6900024
Distancia desde SDQ: 22752.66 km
--------------------------------------------------
Código: DCN
Nombre: RAAF Base Curtin
Ciudad: Derby
País: Australia
Latitud: -17.58139992
Longitud: 123.8280029
Distancia desde SDQ: 21921.62 km
--------------------------------------------------
Código: MKQ


PUNTO 4


In [ ]:
import pandas as pd
import numpy as np
import folium
from heapq import heappop, heappush
from collections import defaultdict

# Función para calcular la distancia entre dos puntos geográficos (Haversine)
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Radio de la Tierra en kilómetros
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

# Crear grafo desde la lista de aristas
def create_graph_from_edges(edge_list):
    graph = defaultdict(list)
    for weight, start, end in edge_list:
        graph[start].append((end, weight))
        graph[end].append((start, weight))
    return graph

# Algoritmo de Dijkstra para encontrar el camino más corto
def dijkstra_shortest_path(graph, start, end):
    distances = {vertex: float('inf') for vertex in graph}
    distances[start] = 0
    previous = {vertex: None for vertex in graph}
    pq = [(0, start)]  # Cola de prioridad

    while pq:
        current_distance, current_vertex = heappop(pq)

        if current_vertex == end:
            break

        if current_distance > distances[current_vertex]:
            continue

        for neighbor, weight in graph[current_vertex]:
            distance = current_distance + weight
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                previous[neighbor] = current_vertex
                heappush(pq, (distance, neighbor))

    # Reconstruir el camino más corto
    path = []
    current = end
    while current is not None:
        path.append(current)
        current = previous[current]
    path.reverse()
    return path, distances[end]

# Cargar datos del CSV
df = pd.read_csv('flights_final.csv')

# Crear un mapeo de códigos de aeropuerto a enteros
airport_codes = pd.concat([df['Source Airport Code'], df['Destination Airport Code']]).unique()
code_to_int = {code: idx for idx, code in enumerate(airport_codes)}
int_to_code = {idx: code for code, idx in code_to_int.items()}

# Crear la lista de aristas y el grafo
edges = []
airport_info = {}

for index, row in df.iterrows():
    source_code = row['Source Airport Code']
    dest_code = row['Destination Airport Code']
    source_lat = row['Source Airport Latitude']
    source_lon = row['Source Airport Longitude']
    dest_lat = row['Destination Airport Latitude']
    dest_lon = row['Destination Airport Longitude']

    # Calcular la distancia usando la fórmula Haversine
    distance = haversine_distance(source_lat, source_lon, dest_lat, dest_lon)

    # Convertir códigos a enteros y agregar arista
    source_int = code_to_int[source_code]
    dest_int = code_to_int[dest_code]
    edges.append((distance, source_int, dest_int))

    # Guardar la información del aeropuerto si no está ya registrada
    if source_code not in airport_info:
        airport_info[source_code] = {
            'name': row['Source Airport Name'],
            'city': row['Source Airport City'],
            'country': row['Source Airport Country'],
            'lat': source_lat,
            'lon': source_lon
        }

    if dest_code not in airport_info:
        airport_info[dest_code] = {
            'name': row['Destination Airport Name'],
            'city': row['Destination Airport City'],
            'country': row['Destination Airport Country'],
            'lat': dest_lat,
            'lon': dest_lon
        }

# Crear el grafo con las aristas
graph = create_graph_from_edges(edges)

# Solicitar los códigos de los aeropuertos de inicio y fin
start_airport_code = input("Ingrese el código del aeropuerto de inicio: ").strip()
end_airport_code = input("Ingrese el código del aeropuerto de destino: ").strip()

# Convertir los códigos a enteros
start_airport_int = code_to_int.get(start_airport_code)
end_airport_int = code_to_int.get(end_airport_code)

if start_airport_int is None or end_airport_int is None:
    print("Código de aeropuerto no válido.")
else:
    # Encontrar el camino más corto entre los aeropuertos
    path, total_distance = dijkstra_shortest_path(graph, start_airport_int, end_airport_int)

    if path:
        print(f"El camino más corto entre {start_airport_code} y {end_airport_code} es:")
        for airport_int in path:
            airport_code = int_to_code[airport_int]
            info = airport_info[airport_code]
            # Mostrar toda la información del aeropuerto
            print(f" - {airport_code} ({info['name']})")
            print(f"   Ciudad: {info['city']}, País: {info['country']}")
            print(f"   Latitud: {info['lat']}, Longitud: {info['lon']}")
            print("-" * 50)

        print(f"Distancia total: {total_distance:.2f} km")

        # Visualizar el camino en un mapa usando folium
        map_path = folium.Map(location=[0, 0], zoom_start=2)
        coords = []
        for airport_int in path:
            airport_code = int_to_code[airport_int]
            info = airport_info[airport_code]
            coords.append((info['lat'], info['lon']))
            folium.Marker(
                location=[info['lat'], info['lon']],
                popup=(f"{airport_code}: {info['name']} ({info['city']}, {info['country']})")
            ).add_to(map_path)

        folium.PolyLine(coords, color="blue", weight=2.5, opacity=1).add_to(map_path)

        # Guardar el mapa
        map_path.save("shortest_path_map.html")
        print("El camino más corto ha sido guardado en 'shortest_path_map.html'.")
    else:
        print(f"No se encontró un camino entre {start_airport_code} y {end_airport_code}.")


Ingrese el código del aeropuerto de inicio: SDQ
Ingrese el código del aeropuerto de destino: CCK
El camino más corto entre SDQ y CCK es:
 - SDQ (Las Américas International Airport)
   Ciudad: Santo Domingo, País: Dominican Republic
   Latitud: 18.42970085, Longitud: -69.66889954
--------------------------------------------------
 - MIA (Miami International Airport)
   Ciudad: Miami, País: United States
   Latitud: 25.79319954, Longitud: -80.29060364
--------------------------------------------------
 - DFW (Dallas Fort Worth International Airport)
   Ciudad: Dallas-Fort Worth, País: United States
   Latitud: 32.896801, Longitud: -97.038002
--------------------------------------------------
 - BNE (Brisbane International Airport)
   Ciudad: Brisbane, País: Australia
   Latitud: -27.38419914, Longitud: 153.1170044
--------------------------------------------------
 - PER (Perth International Airport)
   Ciudad: Perth, País: Australia
   Latitud: -31.94029999, Longitud: 115.9670029
------